#### **Source**: Storage Account
#### **Destination**: Lakehouse Table

In [16]:
# Declaring the path for the delta file
source_data_folder = "Files/streamingdata/streamingfiles"
table_delta_file_location = f"Tables/Struc_streaming_csv_data"
checkpointLocation = f"Files/_checkpoint/Struc_streaming_csv_data"

StatementMeta(, 15306ca0-44fb-4451-829f-1f2b7b459fa2, 18, Finished, Available)

In [17]:
# delete the delta file if it exists ( This isonly reequired if we would like to restart the job from the begingin)
if mssparkutils.fs.exists(table_delta_file_location):
  mssparkutils.fs.rm(table_delta_file_location, recurse=True)
if mssparkutils.fs.exists(checkpointLocation):
  mssparkutils.fs.rm(checkpointLocation, recurse=True)

StatementMeta(, 15306ca0-44fb-4451-829f-1f2b7b459fa2, 19, Finished, Available)

In [18]:
# Read all the csv files written atomically in a directory
from pyspark.sql.types import * 
userSchema = StructType().add("Name", "string").add("Age", "integer").add("Country", "string")
csvDF = spark \
    .readStream \
    .option("sep", ",") \
    .schema(userSchema) \
    .csv(source_data_folder) \
     # Equivalent to format("csv").load("/path/to/directory")

StatementMeta(, 15306ca0-44fb-4451-829f-1f2b7b459fa2, 20, Finished, Available)

In [22]:
from  pyspark.sql.functions import input_file_name, col , element_at , split
from pyspark.sql.functions import current_timestamp
df = csvDF.withColumn("fileName", element_at(split(input_file_name(), "/"), -1)) \
            .withColumn("current_timestamp",current_timestamp())

StatementMeta(, 15306ca0-44fb-4451-829f-1f2b7b459fa2, 24, Finished, Available)

In [23]:
def write2table(df2, epoch_id):
    df2.write.format("delta").mode("append").save(f"Tables/Struc_streaming_csv_data")

# here paritionby partition the data in disk

StatementMeta(, 15306ca0-44fb-4451-829f-1f2b7b459fa2, 25, Finished, Available)

In [24]:
# We are microbatching in every 5 seconds.
df.writeStream \
    .outputMode("append") \
    .trigger(processingTime='10 seconds') \
    .option("checkpointLocation",checkpointLocation) \
    .foreachBatch(write2table) \
    .start() \
    .awaitTermination()

StatementMeta(, 15306ca0-44fb-4451-829f-1f2b7b459fa2, 26, Finished, Cancelled)